# Goud Chain API Testing Suite

Comprehensive test notebook for all Goud Chain API endpoints with timing metrics and reporting.

**Features:**
- Create API keys and authenticate
- Bulk data submission (configurable batch size)
- Collection management and decryption
- Blockchain explorer and analytics
- Performance metrics with visualizations
- Final report generation

## 1. Configuration

Adjust these settings before running tests.

In [ ]:
# ============ Configuration ============

# API Endpoint (Docker internal network or external)
BASE_URL = "http://nginx:8080"  # Use this when running in Docker
# BASE_URL = "http://localhost:8080"  # Use this for external testing

# Test Parameters
BULK_SUBMISSION_COUNT = 100  # Number of blocks to create in bulk test
API_TIMEOUT = 30  # Request timeout in seconds
RETRY_ATTEMPTS = 3  # Number of retries for failed requests

# Debug Settings
VERBOSE_LOGGING = True  # Print detailed logs
SHOW_REQUEST_BODIES = False  # Print full request/response bodies

# Report Settings
EXPORT_TO_CSV = True  # Export results to CSV
EXPORT_TO_JSON = True  # Export results to JSON
GENERATE_CHARTS = True  # Generate timing charts
REPORT_OUTPUT_DIR = "../scratch/"  # Where to save reports

print("✅ Configuration loaded")
print(f"📡 API Endpoint: {BASE_URL}")
print(f"🔢 Bulk submission count: {BULK_SUBMISSION_COUNT}")

## 2. Setup & Utilities

Import libraries and define helper functions.

In [ ]:
import requests
import json
import time
from datetime import datetime
from typing import Dict, List, Any, Optional
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Global state storage
GLOBAL_STATE = {
    "api_key": None,
    "account_id": None,
    "session_token": None,
    "collection_ids": [],
    "test_results": [],
    "timing_data": {},
}

# Configure plotting style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported")
print("✅ Global state initialized")

In [ ]:
# ============ Helper Functions ============

class APIClient:
    """Wrapper for Goud Chain API calls with timing and error handling"""
    
    def __init__(self, base_url: str, timeout: int = 30):
        self.base_url = base_url.rstrip('/')
        self.timeout = timeout
        self.session = requests.Session()
    
    def _log(self, message: str, level: str = "INFO"):
        if VERBOSE_LOGGING or level == "ERROR":
            timestamp = datetime.now().strftime("%H:%M:%S.%f")[:-3]
            print(f"[{timestamp}] {level}: {message}")
    
    def request(self, method: str, endpoint: str, data: Optional[Dict] = None, 
                headers: Optional[Dict] = None) -> tuple[Optional[Dict], float, bool]:
        """Make HTTP request with timing. Returns (response_data, elapsed_time, success)"""
        url = f"{self.base_url}{endpoint}"
        headers = headers or {}
        
        if data:
            headers["Content-Type"] = "application/json"
        
        start_time = time.time()
        
        try:
            self._log(f"{method} {endpoint}")
            
            if method == "GET":
                response = self.session.get(url, headers=headers, timeout=self.timeout)
            elif method == "POST":
                response = self.session.post(url, json=data, headers=headers, timeout=self.timeout)
            else:
                raise ValueError(f"Unsupported method: {method}")
            
            elapsed = time.time() - start_time
            
            if response.status_code >= 200 and response.status_code < 300:
                result = response.json() if response.content else {}
                self._log(f"✅ Success ({response.status_code}) in {elapsed:.3f}s")
                return result, elapsed, True
            else:
                self._log(f"❌ Failed ({response.status_code}): {response.text}", "ERROR")
                return {"error": response.text}, elapsed, False
        
        except Exception as e:
            elapsed = time.time() - start_time
            self._log(f"❌ Exception: {str(e)}", "ERROR")
            return {"error": str(e)}, elapsed, False

def record_test_result(test_name: str, success: bool, elapsed: float, details: Dict = None):
    """Record test result for final report"""
    result = {
        "test_name": test_name,
        "success": success,
        "elapsed_seconds": elapsed,
        "timestamp": datetime.now().isoformat(),
        "details": details or {}
    }
    GLOBAL_STATE["test_results"].append(result)
    
    # Track timing by endpoint
    if test_name not in GLOBAL_STATE["timing_data"]:
        GLOBAL_STATE["timing_data"][test_name] = []
    GLOBAL_STATE["timing_data"][test_name].append(elapsed)

def print_section(title: str):
    """Print formatted section header"""
    print(f"\n{'='*60}")
    print(f"  {title}")
    print(f"{'='*60}\n")

# Initialize API client
api = APIClient(BASE_URL, API_TIMEOUT)

print("✅ Helper functions defined")
print("✅ API client initialized")

## 3. Test: Create Account

Create a new user account and obtain an API key.

In [ ]:
print_section("Test 1: Create Account")

request_data = {
    "metadata": "Test account created from Jupyter notebook"
}

response, elapsed, success = api.request("POST", "/account/create", data=request_data)

if success:
    GLOBAL_STATE["api_key"] = response.get("api_key")
    GLOBAL_STATE["account_id"] = response.get("account_id")
    
    print(f"\n✅ Account created successfully!")
    print(f"   Account ID: {GLOBAL_STATE['account_id']}")
    print(f"   API Key: {GLOBAL_STATE['api_key'][:20]}...")
    print(f"\n⚠️  {response.get('warning')}")
else:
    print(f"\n❌ Failed to create account: {response}")

record_test_result("Create Account", success, elapsed, response)

## 4. Test: Login

Login with API key to obtain a session token.

In [ ]:
print_section("Test 2: Login")

if not GLOBAL_STATE["api_key"]:
    print("❌ No API key available. Run the Create Account test first.")
else:
    request_data = {
        "api_key": GLOBAL_STATE["api_key"]
    }
    
    response, elapsed, success = api.request("POST", "/account/login", data=request_data)
    
    if success:
        GLOBAL_STATE["session_token"] = response.get("session_token")
        
        print(f"\n✅ Login successful!")
        print(f"   Session Token: {GLOBAL_STATE['session_token'][:30]}...")
        print(f"   Expires in: {response.get('expires_in')} seconds")
    else:
        print(f"\n❌ Login failed: {response}")
    
    record_test_result("Login", success, elapsed, response)

## 5. Test: Submit Data (Single)

Submit a single encrypted collection to the blockchain.

In [ ]:
print_section("Test 3: Submit Data (Single)")

if not GLOBAL_STATE["api_key"]:
    print("❌ No API key available. Run the Create Account test first.")
else:
    request_data = {
        "label": "Test Collection",
        "data": json.dumps({"message": "Hello from Jupyter!", "timestamp": time.time()})
    }
    
    headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
    
    response, elapsed, success = api.request("POST", "/data/submit", data=request_data, headers=headers)
    
    if success:
        collection_id = response.get("collection_id")
        GLOBAL_STATE["collection_ids"].append(collection_id)
        
        print(f"\n✅ Data submitted successfully!")
        print(f"   Collection ID: {collection_id}")
        print(f"   Block Number: {response.get('block_number')}")
        print(f"   Message: {response.get('message')}")
    else:
        print(f"\n❌ Data submission failed: {response}")
    
    record_test_result("Submit Data (Single)", success, elapsed, response)

## 6. Test: Bulk Data Submission

Submit multiple collections to test throughput and performance.

In [ ]:
print_section(f"Test 4: Bulk Data Submission ({BULK_SUBMISSION_COUNT} blocks)")

if not GLOBAL_STATE["api_key"]:
    print("❌ No API key available. Run the Create Account test first.")
else:
    headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
    
    start_time = time.time()
    successful_submissions = 0
    failed_submissions = 0
    submission_times = []
    
    print(f"Submitting {BULK_SUBMISSION_COUNT} collections...\n")
    
    for i in range(BULK_SUBMISSION_COUNT):
        request_data = {
            "label": f"Bulk Test #{i+1}",
            "data": json.dumps({
                "iteration": i+1,
                "timestamp": time.time(),
                "test_type": "bulk_submission"
            })
        }
        
        response, elapsed, success = api.request("POST", "/data/submit", data=request_data, headers=headers)
        submission_times.append(elapsed)
        
        if success:
            successful_submissions += 1
            GLOBAL_STATE["collection_ids"].append(response.get("collection_id"))
        else:
            failed_submissions += 1
        
        # Progress indicator
        if (i + 1) % 10 == 0 or i == BULK_SUBMISSION_COUNT - 1:
            progress = (i + 1) / BULK_SUBMISSION_COUNT * 100
            print(f"Progress: {i+1}/{BULK_SUBMISSION_COUNT} ({progress:.1f}%) - "
                  f"Success: {successful_submissions}, Failed: {failed_submissions}")
    
    total_elapsed = time.time() - start_time
    avg_time = sum(submission_times) / len(submission_times) if submission_times else 0
    throughput = BULK_SUBMISSION_COUNT / total_elapsed if total_elapsed > 0 else 0
    
    print(f"\n{'='*60}")
    print(f"📊 Bulk Submission Summary:")
    print(f"   Total submissions: {BULK_SUBMISSION_COUNT}")
    print(f"   Successful: {successful_submissions}")
    print(f"   Failed: {failed_submissions}")
    print(f"   Success rate: {successful_submissions/BULK_SUBMISSION_COUNT*100:.1f}%")
    print(f"   Total time: {total_elapsed:.2f}s")
    print(f"   Average time per submission: {avg_time:.3f}s")
    print(f"   Throughput: {throughput:.2f} submissions/second")
    print(f"{'='*60}")
    
    record_test_result("Bulk Data Submission", successful_submissions == BULK_SUBMISSION_COUNT, 
                      total_elapsed, {
                          "total": BULK_SUBMISSION_COUNT,
                          "successful": successful_submissions,
                          "failed": failed_submissions,
                          "avg_time": avg_time,
                          "throughput": throughput
                      })

## 7. Test: List Collections

Retrieve all collections for the authenticated user.

In [ ]:
print_section("Test 5: List Collections")

if not GLOBAL_STATE["api_key"]:
    print("❌ No API key available. Run the Create Account test first.")
else:
    headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
    
    response, elapsed, success = api.request("GET", "/data/list", headers=headers)
    
    if success:
        collections = response.get("collections", [])
        
        print(f"\n✅ Retrieved {len(collections)} collections")
        
        if collections:
            # Display as DataFrame
            df = pd.DataFrame(collections)
            display(df.head(10))
            
            if len(collections) > 10:
                print(f"\n... and {len(collections) - 10} more")
    else:
        print(f"\n❌ Failed to list collections: {response}")
    
    record_test_result("List Collections", success, elapsed, {"count": len(collections) if success else 0})

## 8. Test: Decrypt Collection

Decrypt a specific collection to verify data integrity.

In [ ]:
print_section("Test 6: Decrypt Collection")

if not GLOBAL_STATE["api_key"]:
    print("❌ No API key available. Run the Create Account test first.")
elif not GLOBAL_STATE["collection_ids"]:
    print("❌ No collections available. Run the Submit Data test first.")
else:
    # Decrypt the first collection
    collection_id = GLOBAL_STATE["collection_ids"][0]
    headers = {"Authorization": f"Bearer {GLOBAL_STATE['api_key']}"}
    
    response, elapsed, success = api.request("POST", f"/data/decrypt/{collection_id}", headers=headers)
    
    if success:
        print(f"\n✅ Collection decrypted successfully!")
        print(f"   Collection ID: {response.get('collection_id')}")
        print(f"   Label: {response.get('label')}")
        print(f"   Created at: {response.get('created_at')}")
        print(f"   Data: {response.get('data')}")
    else:
        print(f"\n❌ Decryption failed: {response}")
    
    record_test_result("Decrypt Collection", success, elapsed, response)

## 9. Test: View Blockchain

Retrieve the entire blockchain state.

In [ ]:
print_section("Test 7: View Blockchain")

response, elapsed, success = api.request("GET", "/chain")

if success:
    chain_length = len(response.get("chain", []))
    node_id = response.get("node_id", "unknown")
    
    print(f"\n✅ Blockchain retrieved successfully!")
    print(f"   Node ID: {node_id}")
    print(f"   Chain length: {chain_length} blocks")
    
    if chain_length > 0:
        latest_block = response["chain"][-1]
        print(f"   Latest block index: {latest_block.get('index')}")
        print(f"   Latest block timestamp: {latest_block.get('timestamp')}")
else:
    print(f"\n❌ Failed to retrieve blockchain: {response}")

record_test_result("View Blockchain", success, elapsed, {"chain_length": chain_length if success else 0})

## 10. Test: Chain Statistics

Get analytics and statistics about the blockchain.

In [ ]:
print_section("Test 8: Chain Statistics")

response, elapsed, success = api.request("GET", "/stats")

if success:
    print(f"\n✅ Statistics retrieved successfully!")
    print(f"   Total blocks: {response.get('total_blocks')}")
    print(f"   Total collections: {response.get('total_collections')}")
    print(f"   Total accounts: {response.get('total_accounts')}")
    print(f"   Avg block time: {response.get('avg_block_time_seconds', 0):.3f}s")
    
    validator_dist = response.get('validator_distribution', {})
    if validator_dist:
        print(f"\n   Validator distribution:")
        for validator, count in validator_dist.items():
            print(f"      {validator}: {count} blocks")
else:
    print(f"\n❌ Failed to retrieve statistics: {response}")

record_test_result("Chain Statistics", success, elapsed, response)

## 11. Test: Node Metrics

Get performance metrics from the current node.

In [ ]:
print_section("Test 9: Node Metrics")

response, elapsed, success = api.request("GET", "/metrics")

if success:
    print(f"\n✅ Metrics retrieved successfully!")
    print(f"   Node ID: {response.get('node_id')}")
    print(f"   Chain length: {response.get('chain_length')}")
    print(f"   Peer count: {response.get('peer_count')}")
    print(f"   Latest block index: {response.get('latest_block_index')}")
    print(f"   Latest block timestamp: {response.get('latest_block_timestamp')}")
    print(f"   Status: {response.get('status')}")
else:
    print(f"\n❌ Failed to retrieve metrics: {response}")

record_test_result("Node Metrics", success, elapsed, response)

## 12. Test: P2P Peers

View connected P2P peers and their reputation.

In [ ]:
print_section("Test 10: P2P Peers")

response, elapsed, success = api.request("GET", "/peers")

if success:
    peers = response.get('peers', [])
    reputation = response.get('reputation', {})
    
    print(f"\n✅ Peer information retrieved successfully!")
    print(f"   Connected peers: {response.get('count', 0)}")
    
    if peers:
        print(f"\n   Peer list:")
        for peer in peers:
            rep = reputation.get(peer, 0)
            print(f"      {peer} (reputation: {rep})")
    else:
        print(f"   No peers connected")
else:
    print(f"\n❌ Failed to retrieve peers: {response}")

record_test_result("P2P Peers", success, elapsed, response)

## 13. Test: Health Check

Verify node health status (used by load balancer).

In [ ]:
print_section("Test 11: Health Check")

response, elapsed, success = api.request("GET", "/health")

if success:
    print(f"\n✅ Node is healthy!")
    print(f"   Status: {response.get('status')}")
    print(f"   Node ID: {response.get('node_id')}")
    print(f"   Chain length: {response.get('chain_length')}")
    print(f"   Peer count: {response.get('peer_count')}")
    print(f"   Latest block: {response.get('latest_block')}")
else:
    print(f"\n❌ Health check failed: {response}")

record_test_result("Health Check", success, elapsed, response)

## 14. Final Report & Visualizations

Generate comprehensive report with timing analysis and charts.

In [ ]:
print_section("Final Report & Analysis")

# Calculate summary statistics
total_tests = len(GLOBAL_STATE["test_results"])
successful_tests = sum(1 for r in GLOBAL_STATE["test_results"] if r["success"])
failed_tests = total_tests - successful_tests
success_rate = (successful_tests / total_tests * 100) if total_tests > 0 else 0

total_time = sum(r["elapsed_seconds"] for r in GLOBAL_STATE["test_results"])
avg_time = total_time / total_tests if total_tests > 0 else 0

print(f"\n📊 Test Execution Summary:")
print(f"{'='*60}")
print(f"Total tests: {total_tests}")
print(f"Successful: {successful_tests}")
print(f"Failed: {failed_tests}")
print(f"Success rate: {success_rate:.1f}%")
print(f"Total execution time: {total_time:.2f}s")
print(f"Average time per test: {avg_time:.3f}s")
print(f"{'='*60}\n")

# Create results DataFrame
df_results = pd.DataFrame(GLOBAL_STATE["test_results"])
print("\n📋 Detailed Results:")
display(df_results[["test_name", "success", "elapsed_seconds", "timestamp"]])

# Export to CSV if configured
if EXPORT_TO_CSV:
    csv_path = f"{REPORT_OUTPUT_DIR}test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(csv_path, index=False)
    print(f"\n✅ Results exported to: {csv_path}")

# Export to JSON if configured
if EXPORT_TO_JSON:
    json_path = f"{REPORT_OUTPUT_DIR}test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(json_path, 'w') as f:
        json.dump({
            "summary": {
                "total_tests": total_tests,
                "successful": successful_tests,
                "failed": failed_tests,
                "success_rate": success_rate,
                "total_time": total_time,
                "avg_time": avg_time
            },
            "results": GLOBAL_STATE["test_results"],
            "global_state": {
                "account_id": GLOBAL_STATE["account_id"],
                "collection_count": len(GLOBAL_STATE["collection_ids"])
            }
        }, f, indent=2)
    print(f"✅ Results exported to: {json_path}")

In [ ]:
# Generate timing charts
if GENERATE_CHARTS and GLOBAL_STATE["timing_data"]:
    print("\n📊 Generating performance charts...\n")
    
    # Chart 1: Average latency per endpoint
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    endpoints = list(GLOBAL_STATE["timing_data"].keys())
    avg_times = [sum(times)/len(times) for times in GLOBAL_STATE["timing_data"].values()]
    
    ax1.barh(endpoints, avg_times, color='steelblue')
    ax1.set_xlabel('Average Response Time (seconds)')
    ax1.set_title('Average Latency by Endpoint')
    ax1.grid(axis='x', alpha=0.3)
    
    # Chart 2: Response time distribution
    all_times = []
    all_labels = []
    for endpoint, times in GLOBAL_STATE["timing_data"].items():
        all_times.extend(times)
        all_labels.extend([endpoint] * len(times))
    
    df_times = pd.DataFrame({'Endpoint': all_labels, 'Time': all_times})
    
    # Box plot for distribution
    sns.boxplot(data=df_times, y='Endpoint', x='Time', ax=ax2)
    ax2.set_xlabel('Response Time (seconds)')
    ax2.set_title('Response Time Distribution')
    ax2.grid(axis='x', alpha=0.3)
    
    plt.tight_layout()
    
    # Save chart
    chart_path = f"{REPORT_OUTPUT_DIR}performance_chart_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    plt.savefig(chart_path, dpi=150, bbox_inches='tight')
    print(f"✅ Chart saved to: {chart_path}")
    
    plt.show()

print("\n✅ Report generation complete!")

## Summary

All tests completed! Check the cells above for detailed results and the `/scratch` directory for exported reports.

**Key Artifacts:**
- API Key: Stored in `GLOBAL_STATE["api_key"]`
- Collection IDs: Stored in `GLOBAL_STATE["collection_ids"]`
- Test Results: Exported to CSV/JSON in `/scratch`
- Performance Charts: Saved in `/scratch`